In [ ]:
# выбор видеокарты
# оставляет видимой только одну GPU 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import torch
# проверка на количество видеокарт (если 1 то все ок, если 2 то надо смотреть)
print('CUDA_VISIBLE_DEVICES', torch.cuda.device_count())
print('CURRENT DEVICE', torch.cuda.current_device())
print('____________________________')

In [ ]:
print(torch.cuda.memory_summary())

In [ ]:
import torch
from longformer.longformer import Longformer, LongformerConfig
from longformer.sliding_chunks import pad_to_window_size
from transformers import RobertaTokenizer

import pandas as pd

In [ ]:
config = LongformerConfig.from_pretrained('../models/longformer-base-4096/') 
config.attention_mode = 'sliding_chunks'

model = Longformer.from_pretrained('../models/longformer-base-4096/', config=config)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# tokenizer.model_max_length = model.config.max_position_embeddings
tokenizer.model_max_length = 200000

model = model.cuda();

In [ ]:
# токенизируем текст
# text - текст в формате строки
# add_special_tokens - True or False, добавлять специальные токены или нет (<cls> <s>)
# НА ВЫХОД:
# tokens - список из токенов
# pad_token_id - токен необходимый в модели
def tokenize_text(text, add_special_tokens, tokenizer):
    if add_special_tokens == True:
        tokens = tokenizer.encode(text, add_special_tokens=True)
    else:
        tokens = tokenizer.encode(text, add_special_tokens=False)
    return tokens, tokenizer.pad_token_id

# сокращение списка токенов до списка <= 4096 
# tokens_list - список токенов 
# new_size - размерность списка, до которого надо сократить
# НА ВЫХОД:
# new_tokens - обрезанный список с токенами (<= new_size)
#     выглядит в формате вложенного списка [[0,1,2]]
def cut_tokens(tokens_list, new_size):
    new_tokens = []
    tok = tokens_list[:new_size]
    new_tokens.append(tok)
    return new_tokens

In [ ]:
# открыть файл
# id - id файла
# folder - путь к папке ('text')
# на ВЫХОД:
# text_in_string текст в формате строки
def open_file(d_id, folder):
    path = folder + '/' + str(d_id) + '.txt'
    text_file = open(path,'r')
    text_in_string = text_file.read()
    text_file.close()
    return text_in_string

In [ ]:
# формируем список пар текстов для обучения модели
# path_dev_file - путь до файла dev.csv
# НА ВЫХОД
# train_dl - список текстов
# (4090 токенов текста1, 4090 токенов текста2, label)
def create_data_for_training(path_dev_file, texts_folder):
    model_size = 2000
    dev_df = pd.read_csv(path_dev_file)
    train_dl = []
    len_dev_df = len(dev_df)
    print("Text preprocessing")
    for i in dev_df.index:
#     i = 0
#     while i < 5:
        doc_id_1 = dev_df.take([i]).values[0][0] #doc_id_1
        doc_id_2 = dev_df.take([i]).values[0][2] #doc_id_2
        label = dev_df.take([i]).values[0][4] #label
        
#         text1 = open_file(doc_id_1, texts_folder)
#         text2 = open_file(doc_id_2, texts_folder)
#         elem = (text1, text2, label)
        
        text1 = open_file(doc_id_1, texts_folder)
        tokens1, pad_token_id1 = tokenize_text(text1, True, tokenizer)
        cut_tok1 = cut_tokens(tokens1, model_size)
#         input_ids1 = torch.tensor(cut_tok1).unsqueeze(0)
#         text_cut1 = tokenizer.decode(input_ids1[0][0])
        
        text2 = open_file(doc_id_2, texts_folder)
        tokens2, pad_token_id2 = tokenize_text(text2, True, tokenizer)
        cut_tok2 = cut_tokens(tokens2, model_size)
#         input_ids2 = torch.tensor(cut_tok2).unsqueeze(0)
#         text_cut2 = tokenizer.decode(input_ids2[0][0])
        
#         elem = (text_cut1, text_cut2, label)
        elem = (cut_tok1[0], cut_tok2[0], label)
#         elem = (input_ids1[0], input_ids2[0], label)
        train_dl.append(elem)
#         i += 1
        
#         time.sleep(0.0000001)
        print('\r', 'iter... {} / {}'.format(str(i), len_dev_df - 1, end=''))
    
    return train_dl

In [ ]:
# amount_label - сколько экземпляров 0 и сколько экземпляров 1 надо добавить в тренировочную базу
def create_data_for_training1(path_dev_file, texts_folder, amount_label):
    model_size = 2000
    df = pd.read_csv(path_dev_file)
    df0 = df[df['label'] == 0]
    df1 = df[df['label'] == 1]
    train_dl = []
    print("Text preprocessing")
    counter = 0
    for i in range(0, len(df0), len(df0)//amount_label):
        if len(train_dl) >= amount_label:
            break
        doc_id_1 = df0.take([i]).values[0][0] #doc_id_1
        doc_id_2 = df0.take([i]).values[0][2] #doc_id_2
        label = df0.take([i]).values[0][4] #label
        
        text1 = open_file(doc_id_1, texts_folder)
        tokens1, pad_token_id1 = tokenize_text(text1, True, tokenizer)
        cut_tok1 = cut_tokens(tokens1, model_size)
        text2 = open_file(doc_id_2, texts_folder)
        tokens2, pad_token_id2 = tokenize_text(text2, True, tokenizer)
        cut_tok2 = cut_tokens(tokens2, model_size)
        
        elem = (cut_tok1[0], cut_tok2[0], label)

        train_dl.append(elem)
        
        counter += 1
        print('\r', 'iter... {} / {}'.format(str(counter), amount_label*2), end='          ')
        
    
    for i in range(0, len(df1), len(df1)//amount_label):
        if len(train_dl) >= amount_label*2:
            break
        doc_id_1 = df1.take([i]).values[0][0] #doc_id_1
        doc_id_2 = df1.take([i]).values[0][2] #doc_id_2
        label = df1.take([i]).values[0][4] #label
        
        text1 = open_file(doc_id_1, texts_folder)
        tokens1, pad_token_id1 = tokenize_text(text1, True, tokenizer)
        cut_tok1 = cut_tokens(tokens1, model_size)
        text2 = open_file(doc_id_2, texts_folder)
        tokens2, pad_token_id2 = tokenize_text(text2, True, tokenizer)
        cut_tok2 = cut_tokens(tokens2, model_size)
        
        elem = (cut_tok1[0], cut_tok2[0], label)

        train_dl.append(elem)

        counter += 1
        print('\r', 'iter... {} / {}'.format(str(counter), amount_label*2), end='          ')
    
    return train_dl

### Предобработка

In [ ]:
# dataset_v1
# path_dev_file = '../data/dataset_v1/dev.csv'
# path_train_file = '../data/dataset_v1/train.csv'
# texts_folder = '../data/dataset_v1/text'

# SimRuWiki
# path_dev_file = '../data/SimRuWiki/dev.csv'
# path_train_file = '../data/SimRuWiki/train.csv'
# texts_folder = '../data/SimRuWiki/texts'

# SimEnWiki
path_dev_file = '../data/SimEnWiki/dev.csv'
path_train_file = '../data/SimEnWiki/train.csv'
texts_folder = '../data/SimEnWiki/texts'

# dev_df = pd.read_csv(path_dev_file)
# train_dl = create_data_for_training(path_train_file, texts_folder)
# train_dl = create_data_for_training(path_dev_file, texts_folder)

amount_label = 1500
train_dl = create_data_for_training1(path_train_file, texts_folder, amount_label)

In [ ]:
# train_dl[:1][0][0]
len(train_dl)

In [ ]:
count0 = 0
count1 = 0
for i, (text1, text2, true_label) in enumerate(train_dl):
    if true_label==0:
        count0 += 1
    elif true_label==1:
        count1 += 1

In [ ]:
count0, count1

In [ ]:
from torch.nn import BCELoss, BCEWithLogitsLoss, Sigmoid, CosineEmbeddingLoss
from torch.optim import SGD
from scipy.spatial.distance import cosine 
import time
import numpy as np
# train the model
def train_model(train_dl, model):
    
#     m = Sigmoid()
    
    # define the optimization
    
    model = model.cuda()
    criterion = BCELoss()
#     criterion = CosineEmbeddingLoss()
    optimizer = SGD(model.parameters(), lr=3e-5, momentum=0.9)
#     model = model.cuda()
    model.train()
    cos = torch.nn.CosineSimilarity(dim=0)
#     model.zero_grad()
    # enumerate epochs
    num_epoch = 1
    for epoch in range(num_epoch):
        print('\nepoch {}/{}'.format(epoch+1, num_epoch))
#         print(torch.cuda.memory_summary())
        for i, (text1, text2, true_label) in enumerate(train_dl):
            
#             temp_cls = torch.tensor([1.])
#             temp_cls = np.array([0.])
            # clear the gradients
            optimizer.zero_grad()
            for j in range(2):
                if j % 2 == 0:
                    # это первый прогон, тут оставляем без градиента
                    input_ids1 = torch.tensor(text1).unsqueeze(0)
                    input_ids1 = input_ids1.cuda()
                    attention_mask1 = torch.ones(input_ids1.shape, dtype=torch.long, device=input_ids1.device) # initialize to local attention
                    attention_mask1[:, [0]] =  2
                    input_ids1, attention_mask1 = pad_to_window_size(input_ids1, attention_mask1, config.attention_window[0], tokenizer.pad_token_id)
                    
                    output1 = model(input_ids1, attention_mask=attention_mask1)[0]
                    cls1 = output1[0][0]
#                     cls1 = torch.reshape(cls1, (1, len(cls1)))
                    temp_cls = cls1.cpu().detach().numpy()
                    # сохраняем этот токен вне цикла
#                     cls11 = torch.tensor(cls1, requires_grad=False)
#                     temp_cls = cls1
                    del output1
                    del cls1
#                     del cls11
                    del input_ids1
                    del attention_mask1
                    torch.cuda.empty_cache()
                    
                else:
                    # это второй прогон, тут градиент оставляем и считаем лосс
                    input_ids2 = torch.tensor(text2).unsqueeze(0)
                    input_ids2 = input_ids2.cuda()
                    attention_mask2 = torch.ones(input_ids2.shape, dtype=torch.long, device=input_ids2.device)
                    attention_mask2[:, [0]] =  2
                    input_ids2, attention_mask2 = pad_to_window_size(input_ids2, attention_mask2, config.attention_window[0], tokenizer.pad_token_id)

                    output2 = model(input_ids2, attention_mask=attention_mask2)[0]
                    cls2 = output2[0][0]
#                     cls2 = torch.reshape(cls2, (1, len(cls2)))
                    del output2
                    torch.cuda.empty_cache()

                    # считаем лосс (сначала косинусное сходство)
#                     temp_cls = temp_cls.to(device=cls2.device)
                    temp_cls = torch.tensor(temp_cls, device=cls2.device)                    
            
#                     делим на два чтобы распределение сделать от 0 до 1
#                     dist = float(1 - cosine(cls1, cls2)) / 2
                    dist = cos(temp_cls, cls2)
#                     print('cls2 size=', cls2.size())
#                     print('dist=', dist)
                    # сравнение с порогом для жесткого штрафа модели (делаем либо 0 либо 1)
                    threshold = torch.tensor(0.7)
# #                     print('thre=', threshold)
                    if dist > threshold: # dist = 1
#                         difference = torch.tensor(1-float(dist))
#                         dist = dist + difference
                        dist = torch.tensor(1., requires_grad = True)
                    else: # dist = 0
# #                         dist = dist - dist
                        dist = torch.tensor(0., requires_grad = True)


#                     loss_input = dist
                    loss_target = torch.tensor(true_label).float()
#                     print('dist=', dist)
#                     print('targe=', loss_target)
                    # without Sigmoid
                    loss = criterion(dist, loss_target) # 0 or 1
                    #with Sigmoid
#                     loss = criterion(m(loss_input), loss_target) # 0 or 1
#                     print('input=', loss_input)
#                     print('targe=', loss_target)
#                     print('loss=', loss)
                    del true_label
                    del temp_cls
                    del input_ids2
                    del attention_mask2
                    del cls2
                    del dist
#                     del loss_input
                    del loss_target
                    torch.cuda.empty_cache()
                    # credit assignment
                    loss.backward()
            
            # update model weights
            optimizer.step()
            
            # вывод процесса итераций в консоль
            print('\r', 'iter... {} / {}, loss = {}'.format(str(i), len(train_dl)-1, loss.item()), end='                   ')
            del loss
            torch.cuda.empty_cache()

In [ ]:
train_model(train_dl, model)

In [ ]:
# сохранение модели в путь save_model_to

# save_model_to = '../models/longformer-base-4096-epoch1-lr3e-5-cosemblos-dataset1/'
# save_model_to = '../models/longformer-base-4096-epoch4-lr3e-5-cosemblos-dataset-train-3500/'
save_model_to = '../models/longformer-base-4096-epoch1-lr3e-5-bceloss-simenwiki-train-on-3000-notdel2/'
# # save_model_to = '../models/longformer-base-4096-epoch1-lr3e-5-cosemblos-dataset2/'
model.save_pretrained(save_model_to)
tokenizer.save_pretrained(save_model_to)

# КОДИРОВКА НАИМЕНОВАНИЯ ТРЕНИРОВАННОЙ МОДЕЛИ
# epoch1 - количество эпох 1+
# lr3e-5 - learning rate 3е-5 
# sigm or nosigm - выбираем с сигмоидой или без
# pen08 - использование жесткого штрафа модели с порогом 0.8

# save_model_to = '../models/longformer-base-4096-epoch1-lr3e-4-sigm-pen08/'
# save_model_to = '../models/longformer-base-4096-epoch1-lr3e-4-sigm/'
# save_model_to = '../models/longformer-base-4096-epoch1-lr3e-5-nosigm/'
# model.save_pretrained(save_model_to)
